In [1]:
#csvファイルの読み込み
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df_test = pd.read_csv(r"C:\Users\eikik\python\python_dat\equity-post-HCT-survival-predictions\test.csv")

In [2]:
#データの確認
df_test.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,28800,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,28802,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0


In [3]:
#データの次元の確認
df_test.shape

(3, 58)

In [3]:
#データの情報
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 58 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      3 non-null      int64  
 1   dri_score               3 non-null      object 
 2   psych_disturb           3 non-null      object 
 3   cyto_score              1 non-null      object 
 4   diabetes                3 non-null      object 
 5   hla_match_c_high        2 non-null      float64
 6   hla_high_res_8          2 non-null      float64
 7   tbi_status              3 non-null      object 
 8   arrhythmia              3 non-null      object 
 9   hla_low_res_6           3 non-null      float64
 10  graft_type              3 non-null      object 
 11  vent_hist               3 non-null      object 
 12  renal_issue             3 non-null      object 
 13  pulm_severe             3 non-null      object 
 14  prim_disease_hct        3 non-null      object

In [22]:
# 最頻値を計算
dri_score_mode = df_test['dri_score'].mode()[0]

# 欠損値を最頻値で補完
df_test['dri_score'] = df_test['dri_score'].fillna(dri_score_mode)

# 補完後の欠損値の確認
missing_after = df_test['dri_score'].isnull().sum()
print(f"補完後の'dri_score'の欠損値数: {missing_after}")


補完後の'dri_score'の欠損値数: 0


In [23]:
#df_testのデータの欠損割合の表示
missing_rate = df_test.isnull().sum() / len(df_test)
#多い順に並べ替え
missing_rate = missing_rate.sort_values(ascending=False)
missing_rate = missing_rate[missing_rate > 0]
missing_rate


tce_imm_match       0.333333
hla_high_res_10     0.333333
hla_high_res_8      0.333333
hla_match_c_high    0.333333
tce_div_match       0.333333
dtype: float64

In [24]:
#欠損の割合が0.5以上の列を削除
df_test = df_test.dropna(thresh=len(df_test)*0.5, axis=1)

In [25]:
#欠損の割合が0.10以下の列を抽出
low_missing_rate = missing_rate[missing_rate < 0.10]
low_missing_rate

Series([], dtype: float64)

In [26]:
#low_missing_rateの列は最頻値で補完
for column in low_missing_rate.index:
    mode = df_test[column].mode()[0]
    df_test[column] = df_test[column].fillna(mode)

ここまでで
欠損値割合が5%以下のデータは最頻値で補完
欠損値割合が50%以上のデータは削除
している。\
間の欠損値のデータについて、以下において、相関係数の高いもの同士で単回帰分析を行い、欠損値の補完を行う。

In [27]:
#更新したdf_testの欠損率の確認
#欠損率が0.5以上のもの、0.1以下のものがないことを確認
missing_rate_2 = df_test.isnull().sum() / len(df_test)
missing_rate_2 = missing_rate.sort_values(ascending=False)
missing_rate_2 = missing_rate[missing_rate > 0]
missing_rate_2


tce_imm_match       0.333333
hla_high_res_10     0.333333
hla_high_res_8      0.333333
hla_match_c_high    0.333333
tce_div_match       0.333333
dtype: float64

In [30]:
# df_testからmissing_rateの列を抽出
missing_rate_columns = missing_rate_2.index

# missing_rate_columnsの列のうち、df_testから同じ列名の列を抽出
df_missing_rate = df_test[missing_rate_columns]

df_missing_rate

,tce_imm_match,hla_high_res_10,hla_high_res_8,hla_match_c_high,tce_div_match
0,NaN,NaN,NaN,NaN,NaN
1,P/P,10.0,8.0,2.0,Permissive mismatched
2,P/P,10.0,8.0,2.0,Permissive mismatched


In [34]:
# 数値型の列のみを選択
numeric_df_test = df_missing_rate.select_dtypes(include=[np.number])

numeric_df_test

,hla_high_res_10,hla_high_res_8,hla_match_c_high
0,NaN,NaN,NaN
1,10.0,8.0,2.0
2,10.0,8.0,2.0


In [36]:
# 変数間の相関係数を計算
correlation = numeric_df_test.corr()
#変数の相関係数が0.8以上の組み合わせを抽出
high_correlation = correlation[correlation > 0.8]
#high_correlationの組み合わせを表示
high_correlation


,hla_high_res_10,hla_high_res_8,hla_match_c_high
hla_high_res_10,NaN,NaN,NaN
hla_high_res_8,NaN,NaN,NaN
hla_match_c_high,NaN,NaN,NaN


In [33]:
for column in high_correlation.columns:
    # 自分自身を除いた相関を取得
    other_corr = high_correlation[column].drop(column)
    max_corr = other_corr.max()
    most_correlated_feature = other_corr.idxmax()  # 最大相関の特徴量を取得
    print(f"{column}と相関が最も高い特徴量: {most_correlated_feature} (相関係数: {max_corr})")


hla_high_res_10と相関が最も高い特徴量: nan (相関係数: nan)
hla_high_res_8と相関が最も高い特徴量: nan (相関係数: nan)
hla_match_c_highと相関が最も高い特徴量: nan (相関係数: nan)


C:\Users\eikik\AppData\Local\Temp\ipykernel_8768\1400108618.py:5: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  most_correlated_feature = other_corr.idxmax()  # 最大相関の特徴量を取得
C:\Users\eikik\AppData\Local\Temp\ipykernel_8768\1400108618.py:5: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  most_correlated_feature = other_corr.idxmax()  # 最大相関の特徴量を取得
C:\Users\eikik\AppData\Local\Temp\ipykernel_8768\1400108618.py:5: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  most_correlated_feature = other_corr.idxmax()  # 最大相関の特徴量を取得


In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# 先に high_correlation の結果を利用して、各特徴量に対して相方を決定
most_correlated = {}
for column in high_correlation.columns:
    other_corr = high_correlation[column].drop(column).dropna()  # 自分自身を除外し NaN を除く
    if other_corr.empty:
        print(f"{column}: 他の特徴量との相関係数が有効な値ではありません（すべて NaN）。スキップします。")
        continue
    most_correlated_feature = other_corr.idxmax()  # 最大相関の特徴量
    most_correlated[column] = most_correlated_feature

# 欠損値の補完処理
for target_column, related_column in most_correlated.items():
    print(f"補完処理: {target_column} を {related_column} を使って補完中")

    # 欠損値がない行をモデルの訓練データとする
    train_data = df_train.dropna(subset=[target_column, related_column])

    # 欠損値がある行を予測対象データとする
    missing_data = df_train[df_train[target_column].isnull()]

    if missing_data.empty:
        print(f"{target_column} に欠損値はありません。スキップします。")
        continue

    # モデルの訓練
    X_train = train_data[[related_column]]
    y_train = train_data[target_column]

    # 欠損値を補完するためのインプーターを作成
    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = imputer.fit_transform(X_train)

    model = LinearRegression()
    model.fit(X_train_imputed, y_train)

    # 欠損値の補完
    X_missing = missing_data[[related_column]]
    X_missing_imputed = imputer.transform(X_missing)
    predicted_values = model.predict(X_missing_imputed)

    # 補完結果をデータに反映
    df_train.loc[missing_data.index, target_column] = predicted_values

# 結果の確認
print("補完後の欠損値数:")
print(df_train.isnull().sum())



hla_match_dqb1_low: 他の特徴量との相関係数が有効な値ではありません（すべて NaN）。スキップします。
補完処理: hla_high_res_10 を hla_high_res_8 を使って補完中
hla_high_res_10 に欠損値はありません。スキップします。
補完処理: hla_high_res_8 を hla_high_res_10 を使って補完中
hla_high_res_8 に欠損値はありません。スキップします。
補完処理: hla_high_res_6 を hla_high_res_8 を使って補完中
hla_high_res_6 に欠損値はありません。スキップします。
補完処理: hla_match_dqb1_high を hla_high_res_10 を使って補完中
hla_match_dqb1_high に欠損値はありません。スキップします。
補完処理: hla_low_res_10 を hla_low_res_8 を使って補完中
hla_low_res_10 に欠損値はありません。スキップします。
補完処理: hla_match_c_high を hla_high_res_8 を使って補完中
hla_match_c_high に欠損値はありません。スキップします。
補完処理: hla_match_a_high を hla_high_res_6 を使って補完中
hla_match_a_high に欠損値はありません。スキップします。
補完処理: hla_nmdp_6 を hla_low_res_10 を使って補完中
hla_nmdp_6 に欠損値はありません。スキップします。
補完処理: hla_match_b_high を hla_high_res_6 を使って補完中
hla_match_b_high に欠損値はありません。スキップします。
補完処理: hla_low_res_8 を hla_low_res_10 を使って補完中
hla_low_res_8 に欠損値はありません。スキップします。
補完処理: hla_match_drb1_high を hla_high_res_6 を使って補完中
hla_match_drb1_high に欠損値はありません。スキップします。
補完処理: hla_low_res_6 を 

In [39]:
#hla_match_dqb1_lowの統計量の表示
df_train['hla_match_dqb1_low'].describe()

count    24606.000000
mean         1.773795
std          0.427130
min          0.000000
25%          2.000000
50%          2.000000
75%          2.000000
max          2.000000
Name: hla_match_dqb1_low, dtype: float64

In [40]:
#hla_match_dqb1_lowを最頻値で補完
mode = df_train['hla_match_dqb1_low'].mode()[0]
df_train['hla_match_dqb1_low'] = df_train['hla_match_dqb1_low'].fillna(mode)


In [41]:
#df_trainの欠損割合の表示
missing_rate_3 = df_train.isnull().sum() / len(df_train)
#多い順に並べ替え
missing_rate_3 = missing_rate_3.sort_values(ascending=False)
missing_rate_3= missing_rate_3[missing_rate_3 > 0]
missing_rate_3


cyto_score_detail         0.413993
tce_div_match             0.395694
tce_imm_match             0.386562
cyto_score                0.280139
conditioning_intensity    0.166285
dtype: float64

In [42]:
#missing_rate_3の列を表示
missing_rate_columns_3 = missing_rate_3.index
df_missing_rate_3 = df_train[missing_rate_columns_3]
df_missing_rate_3

,cyto_score_detail,tce_div_match,tce_imm_match,cyto_score,conditioning_intensity
0,NaN,NaN,NaN,NaN,NaN
1,Intermediate,Permissive mismatched,P/P,Intermediate,MAC
2,NaN,Permissive mismatched,P/P,NaN,NaN
3,Intermediate,Permissive mismatched,P/P,Intermediate,MAC
4,NaN,Permissive mismatched,NaN,NaN,MAC
...,...,...,...,...,...
28795,Intermediate,Bi-directional non-permissive,P/P,Favorable,MAC
28796,TBD,GvH non-permissive,G/G,Poor,RIC
28797,Poor,GvH non-permissive,G/G,Poor,MAC
28798,NaN,Permissive mismatched,P/P,Poor,NMA


In [43]:
#cyto_score_detailの統計量の表示
df_train['cyto_score_detail'].describe()
#要素ごとの個数を表示
df_train['cyto_score_detail'].value_counts()

cyto_score_detail
Intermediate    11158
Poor             3323
Favorable        1208
TBD              1043
Not tested        145
Name: count, dtype: int64

In [44]:
#cyto_score_detailを最頻値で補完
mode = df_train['cyto_score_detail'].mode()[0]
df_train['cyto_score_detail'] = df_train['cyto_score_detail'].fillna(mode) 

In [45]:
#tce_div_matchの統計量の表示
df_train['tce_div_match'].describe()
#要素ごとの個数を表示
df_train['tce_div_match'].value_counts()

tce_div_match
Permissive mismatched            12936
GvH non-permissive                2458
HvG non-permissive                1417
Bi-directional non-permissive      593
Name: count, dtype: int64

In [46]:
#tce_div_matchを最頻値で補完
mode = df_train['tce_div_match'].mode()[0]
df_train['tce_div_match'] = df_train['tce_div_match'].fillna(mode)

In [47]:
#tce_imm_matchの統計量の表示
df_train['tce_imm_match'].describe()
#要素ごとの個数を表示
df_train['tce_imm_match'].value_counts()

tce_imm_match
P/P    13114
G/G     2522
H/H     1084
G/B      544
H/B      229
P/H       83
P/B       66
P/G       25
Name: count, dtype: int64

In [48]:
#tce_imm_matchを最頻値で補完
mode = df_train['tce_imm_match'].mode()[0]
df_train['tce_imm_match'] = df_train['tce_imm_match'].fillna(mode)

In [49]:
#cyto_scoreの統計量の表示
df_train['cyto_score'].describe()
#要素ごとの個数を表示
df_train['cyto_score'].value_counts()

cyto_score
Poor            8802
Intermediate    6376
Favorable       3011
TBD             1341
Normal           643
Other            504
Not tested        55
Name: count, dtype: int64

In [50]:
#cyto_scoreを度数を元に作成した確率を用いて補完
prob = df_train['cyto_score'].value_counts(normalize=True)
df_train['cyto_score'] = df_train['cyto_score'].fillna(np.random.choice(prob.index, p=prob.values))

In [51]:
#conditioning_intensityの統計量の表示
df_train['conditioning_intensity'].describe()
#要素ごとの個数を表示
df_train['conditioning_intensity'].value_counts()

conditioning_intensity
MAC                              12288
RIC                               7722
NMA                               3479
TBD                                373
No drugs reported                   87
N/A, F(pre-TED) not submitted       62
Name: count, dtype: int64

In [52]:
#conditioning_intensityを度数を元に作成した確率を用いて補完
prob = df_train['conditioning_intensity'].value_counts(normalize=True)
df_train['conditioning_intensity'] = df_train['conditioning_intensity'].fillna(np.random.choice(prob.index, p=prob.values))

In [53]:
#train.csvのobject型の列を抽出
object_columns = df_train.select_dtypes(include=object)
object_columns


,dri_score,psych_disturb,cyto_score,diabetes,tbi_status,arrhythmia,graft_type,vent_hist,renal_issue,pulm_severe,...,gvhd_proph,rheum_issue,sex_match,race_group,hepatic_mild,tce_div_match,donor_related,melphalan_dose,cardiac,pulm_moderate
0,N/A - non-malignant indication,No,Poor,No,No TBI,No,Bone marrow,No,No,No,...,FKalone,No,M-F,More than one race,No,Permissive mismatched,Unrelated,"N/A, Mel not given",No,No
1,Intermediate,No,Intermediate,No,"TBI +- Other, >cGy",No,Peripheral blood,No,No,No,...,Other GVHD Prophylaxis,No,F-F,Asian,No,Permissive mismatched,Related,"N/A, Mel not given",No,Yes
2,N/A - non-malignant indication,No,Poor,No,No TBI,No,Bone marrow,No,No,No,...,Cyclophosphamide alone,No,F-M,More than one race,No,Permissive mismatched,Related,"N/A, Mel not given",No,No
3,High,No,Intermediate,No,No TBI,No,Bone marrow,No,No,No,...,FK+ MMF +- others,No,M-M,White,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",No,No
4,High,No,Poor,No,No TBI,No,Peripheral blood,No,No,No,...,TDEPLETION +- other,No,M-F,American Indian or Alaska Native,No,Permissive mismatched,Related,MEL,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Intermediate - TED AML case <missing cytogenetics,No,Favorable,No,No TBI,No,Peripheral blood,No,No,No,...,FK+ MTX +- others(not MMF),No,M-F,More than one race,No,Bi-directional non-permissive,Related,"N/A, Mel not given",No,No
28796,High,No,Poor,Yes,No TBI,No,Peripheral blood,No,No,No,...,Cyclophosphamide +- others,No,M-F,Native Hawaiian or other Pacific Islander,No,GvH non-permissive,Related,"N/A, Mel not given",Yes,Yes
28797,TBD cytogenetics,No,Poor,No,No TBI,No,Peripheral blood,No,No,No,...,FK+ MMF +- others,No,M-F,Native Hawaiian or other Pacific Islander,No,GvH non-permissive,Unrelated,"N/A, Mel not given",No,No
28798,N/A - non-malignant indication,No,Poor,No,No TBI,No,Peripheral blood,No,No,No,...,Cyclophosphamide alone,No,M-M,Black or African-American,No,Permissive mismatched,Related,MEL,No,No


In [54]:
#object_columnsの列をワンホットエンコーディング
df_train = pd.get_dummies(df_train, drop_first=True)

In [55]:
#データの欠損の確認
missing_rate_4 = df_train.isnull().sum() / len(df_train)
missing_rate_4 = missing_rate_4.sort_values(ascending=False)
missing_rate_4 = missing_rate_4[missing_rate_4 > 0]
missing_rate_4

Series([], dtype: float64)

よくわかんないけど、とりあえず欠損値はなくなった！

In [56]:
#データの正規化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns)

In [57]:
#欠損値がなくなり、正規化を行ったdf_trainのデータをCSVファイルに保存
df_train_scaled.to_csv(r"C:\Users\eikik\python\python_dat\train_preprocessed.csv", index=False)

In [ ]:
#df_trainに施した処理をdf_testにも適用
#最頻値で補完

# 最頻値を計算
dri_score_mode = df_test['dri_score'].mode()[0]

# 欠損値を最頻値で補完
df_test['dri_score'] = df_test['dri_score'].fillna(dri_score_mode)

# 補完後の欠損値の確認
missing_after = df_test['dri_score'].isnull().sum()

#欠損の割合が0.5以上の列を削除
df_test = df_test.dropna(thresh=len(df_test)*0.5, axis=1)

#欠損の割合が0.10以下の列を抽出
low_missing_rate = missing_rate[missing_rate < 0.10]

#low_missing_rateの列は最頻値で補完
for column in low_missing_rate.index:
    mode = df_test[column].mode()[0]
    df_test[column] = df_test[column].fillna(mode)

#更新したdf_testの欠損率の確認
#欠損率が0.5以上のもの、0.1以下のものがないことを確認
missing_rate_2 = df_test.isnull().sum() / len(df_test)
missing_rate_2 = missing_rate.sort_values(ascending=False)

# df_testからmissing_rateの列を抽出
missing_rate_columns = missing_rate_2.index

# missing_rate_columnsの列のうち、df_testから同じ列名の列を抽出
df_missing_rate = df_test[missing_rate_columns]

# 数値型の列のみを選択
numeric_df_test = df_missing_rate.select_dtypes(include=[np.number])

# 変数間の相関係数を計算
correlation = numeric_df_test.corr()
#変数の相関係数が0.8以上の組み合わせを抽出
high_correlation = correlation[correlation > 0.8]
#high_correlationの組み合わせを表示
high_correlation

for column in high_correlation.columns:
    # 自分自身を除いた相関を取得
    other_corr = high_correlation[column].drop(column)
    max_corr = other_corr.max()
    most_correlated_feature = other_corr.idxmax()  # 最大相関の特徴量を取得
    print(f"{column}と相関が最も高い特徴量: {most_correlated_feature} (相関係数: {max_corr})")

from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleIm
# 先に high_correlation の結果を利用して、各特徴量に対して相方を決定
most_correlated = {}
for column in high_correlation.columns:
    other_corr = high_correlation[column].drop(column).dropna()  # 自分自身を除外し NaN を除く
    if other_corr.empty:
        print(f"{column}: 他の特徴量との相関係数が有効な値ではありません（すべて NaN）。スキップします。")
        continue
    most_correlated_feature = other_corr.idxmax()  # 最大相関の特徴量
    most_correlated[column] = most_correlated_feature

# 欠損値の補完処理
for target_column, related_column in most_correlated.items():
    print(f"補完処理: {target_column} を {related_column} を使って補完中")

    # 欠損値がない行をモデルの訓練データとする
    train_data = df_test.dropna(subset=[target_column, related_column])

    # 欠損値がある行を予測対象データとする
    missing_data = df_test[df_test[target_column].isnull()]

    if missing_data.empty:
        print(f"{target_column} に欠損値はありません。スキップします。")
        continue

    # モデルの訓練
    X_train = train_data[[related_column]]
    y_train = train_data[target_column]

    # 欠損値を補完するためのインプーターを作成
    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = imputer.fit_transform(X_train)

    model = LinearRegression()
    model.fit(X_train_imputed, y_train)

    # 欠損値の補完
    X_missing = missing_data[[related_column]]
    X_missing_imputed = imputer.transform(X_missing)
    predicted_values = model.predict(X_missing_imputed)

    # 補完結果をデータに反映
    df_test.loc[missing_data.index, target_column] = predicted_values

# 結果の確認
print("補完後の欠損値数:")

#hla_match_dqb1_lowの統計量の表示
df_test['hla_match_dqb1_low'].describe()

#hla_match_dqb1_lowを最頻値で補完
mode = df_test['hla_match_dqb1_low'].mode()[0]
df_test['hla_match_dqb1_low'] = df_test['hla_match_dqb1_low'].fillna(mode)
